In [1]:
from selenium import webdriver #Import the Selenium Webdriver 
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import Select

#Import this functions for the execution of Explicitly Wait.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#Import Pandas
import pandas as pd    



options = Options()
options.headless = True # Make sure that chrome doesn't automatically opens up on it's own.
options.add_argument("--window-size=1920,1200")

#Download the latest chrome-driver from this link (https://sites.google.com/a/chromium.org/chromedriver/home). 
#Make sure that you have an update version of chrome installed on your computer

DRIVER_PATH = '/Users/frankburg/Documents/chromedriver-2'

driver = webdriver.Chrome( options=options, executable_path=DRIVER_PATH)


ModuleNotFoundError: ignored

In [ ]:
#print(driver.page_source)

In [ ]:
#Open up the OPSIN page
driver.get("https://opsin.ch.cam.ac.uk/")
search_chem = driver.find_element_by_id("chemicalName") #Find the chemical
search_chem.clear() # Clear. the input on the form.
search_chem.send_keys("5,5'-dimethyl2,2'-bipyridinyl") #Fill out the box by using send_key function

#driver.find_element_by_xpath('//*[@id="chemicalNameForm"]/fieldset/input[2]').click()
driver.find_element_by_id('chemicalNameForm').submit() #find the submit button and click it.

#We don't need the stdkey again, we will just go with the SMILES only. 
driver.implicitly_wait(10)
stdkey= driver.find_element_by_xpath('//*[@id="stdinchikey"]/a').text
smiles=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "smiles"))).text


In [ ]:
print(stdkey)
print(smiles)

PTRATZCAGVBFIQ-UHFFFAOYSA-N
CC=1C=CC(=NC1)C1=NC=C(C=C1)C


In [ ]:
#smiles=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "smiles"))).text


In [ ]:
#Reading the datasheet from your PC 

data_sheet=pd.read_excel('/Users/frankburg/Downloads/Drug_Data.xlsx')
chemical_name=data_sheet['Chemical Name'] #Grab only the chemical name column
chemical_name=[x.strip() for x in list(chemical_name)] #Remove leading and trailing white space
active_ingred=data_sheet['Active ingredients'] #Drug name
drug_name=data_sheet['Drug Name']

In [ ]:
chemical_name

["5,5'-dimethyl2,2'-bipyridinyl",
 "USP racemic α1 [(tert-Butylamino)methyl]-4-hydroxy-w-xylene- α, α '-diol sulfate (2:l)(salt)"]

In [ ]:
mined_data = []
unmined_data=[]
for i,name in enumerate(chemical_name):
    try:
        driver.get("https://opsin.ch.cam.ac.uk/")
        search_chem = driver.find_element_by_id("chemicalName") #Find the chemical
        search_chem.clear()
        search_chem.send_keys(name) #Fill out the box by using send_key function
        driver.find_element_by_id('chemicalNameForm').submit() #find the submit button and click it.
        
        #We don't need the stdkey again, we will just go with the SMILES only. 
        driver.implicitly_wait(2)
        stdkey= driver.find_element_by_xpath('//*[@id="stdinchikey"]/a').text
        smiles=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "smiles"))).text
        
        smiles_list=[i,drug_name[i],active_ingred[i],name,smiles] #includes the drug name and active ingredients.
        mined_data.append(smiles_list)
    except Exception as e:
        unmined_data.append([i,drug_name[i],active_ingred[i],name])

### INSTRUCTIONS

##### 1. Make the that the chemical name you extracted have no typographical errors or have additonal characters, special character in it, especially at the end. To minimize chances of not mining.  

##### 2. You will create a pandas dataframe for each of mined data and unmined data with their index as unique IDs. 

##### 3. Then we will export them as seperate excel sheet. If possible create a new folder for this on your PC and use the path when exporting below. 
##### 4. Then inspect the unmined data excel and google the for the SMILES of the chemical name. Then as usual copy and paste. The motive is to have all SMILES of the drug before we proceed to mine other property because we would use it as input for mining.

In [ ]:
#Convert to Pandas DataFrame
mined_df = pd.DataFrame(mined_data,columns=['id','Drug Name','Active Ingredient','chemical name', 'SMILES'])

unmined_df = pd.DataFrame(unmined_data,columns=['id','Drug Name','Active Ingredient','chemical name'])


In [ ]:
# Export to Excel
a=mined_df.to_excel('/Users/frankburg/Documents/drugs/mined.xlsx') # I save in a new folder I created called drugs.
b=unmined_df.to_excel('/Users/frankburg/Documents/drugs/unmined.xlsx') 